# CNN

## Fungsi dan Variabel

In [4]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1" 

import matplotlib.pyplot as plt

from Datasets.cifar10 import x_train, y_train, x_val, y_val, x_test, y_test
from Models.CNN.CNN import CNN
from Models.CNN.training.configs import (
    CONV_LAYER_VARIATIONS, FILTER_VARIATIONS, 
    KERNEL_SIZE_VARIATIONS, POOLING_VARIATIONS
)

SAVE_PATH = "Models/CNN/training"

In [11]:
def plot_loss_curves(history, title):
    """Displays validation loss and training loss per epoch"""
    
    plt.plot(history.history["loss"], label = "Train Loss")
    plt.plot(history.history["val_loss"], label = "Validation Loss")
    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid()
    plt.show()

In [12]:
def train_and_evaluate_all(variation_dict, variation_name):
    """Train and evaluate CNN models for each config in variation_dict."""
    
    print(f"\n\n=== {variation_name} Variations ===\n")
    for name, config in variation_dict.items():
        print(f"\n--- Training: {name} ---")

        # Build
        model = CNN.from_config(config)
        model.weights_dir = f"{SAVE_PATH}/weights/{variation_name}"
        model.build()

        # Train & Evaluate
        model.train(x_train, y_train, x_val, y_val)
        y_pred_probs = model.model.predict(x_test)
        model.evaluate(y_pred_probs, y_test)

        # Save weights
        model.save()

        # Log results
        print(f"{name} | Macro F1 Score: {model.f1_score:.4f}")
        plot_loss_curves(model.history, f"{name} | Training & Validation Loss")

## Training

### Jumlah Layer Konvolusi

In [13]:
train_and_evaluate_all(CONV_LAYER_VARIATIONS, "ConvLayerCount")

### Banyak Filter

In [14]:
train_and_evaluate_all(FILTER_VARIATIONS, "FilterSize")

### Ukuran Filter

In [15]:
train_and_evaluate_all(KERNEL_SIZE_VARIATIONS, "KernelSize")

### Tipe Pooling

In [16]:
train_and_evaluate_all(POOLING_VARIATIONS, "PoolingType")

## Forward Propagation

In [1]:
def compare_scratch_keras_cnn():
    # Build CNN
    cnn = CNN.from_config(CONV_LAYER_VARIATIONS["3 Layers"])
    cnn.build()

    # Load Weights
    cnn.load("ConvLayerCount/3layers_filters32-64-128_kernels3-3-3_poolmax.weights.h5")

    # Forward prop
    output_scratch = cnn.forward_scratch(x_test)
    output_keras = cnn.model.predict(x_test)
    
    # Evaluate
    cnn.evaluate(output_scratch, y_test)
    score_scratch = cnn.f1_score

    cnn.evaluate(output_keras, y_test)
    score_keras = cnn.f1_score

    # Output
    print(f"From Scratch | F1-Score: {score_scratch}")
    print(f"Keras | F1-Score: {score_keras}")

In [5]:
compare_scratch_keras_cnn()